In [43]:
import os
# Find the latest version of spark 3.0  from http://www.apache.org/dist/spark/ and enter as the spark version
spark_version = 'spark-3.2.1'
os.environ['SPARK_VERSION']=spark_version

# Install Spark and Java
!apt-get update
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
!wget -q http://www.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop2.7.tgz
!tar xf $SPARK_VERSION-bin-hadoop2.7.tgz
!pip install -q findspark

# Set Environment Variables
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop2.7"

# Start a SparkSession
import findspark
findspark.init()

Hit:1 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease
Ign:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Ign:3 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Get:4 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Hit:5 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release
Hit:6 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Hit:7 http://archive.ubuntu.com/ubuntu bionic InRelease
Hit:8 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease
Get:9 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Hit:10 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Hit:13 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu bionic InRelease
Get:14 http://archive.ubuntu.com/ubuntu bionic-backports InRelease [74.6 kB]
Hit:15 http://ppa.launchpad.net/graph

In [44]:
!wget https://jdbc.postgresql.org/download/postgresql-42.2.9.jar

--2022-02-08 22:40:50--  https://jdbc.postgresql.org/download/postgresql-42.2.9.jar
Resolving jdbc.postgresql.org (jdbc.postgresql.org)... 72.32.157.228, 2001:4800:3e1:1::228
Connecting to jdbc.postgresql.org (jdbc.postgresql.org)|72.32.157.228|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 914037 (893K) [application/java-archive]
Saving to: ‘postgresql-42.2.9.jar.2’

postgresql-42.2.9.j 100%[===================>] 892.61K  2.90MB/s    in 0.3s    

2022-02-08 22:40:51 (2.90 MB/s) - ‘postgresql-42.2.9.jar.2’ saved [914037/914037]



In [45]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("CloudETL").config("spark.driver.extraClassPath","/content/postgresql-42.2.9.jar").getOrCreate()


# EXTRACT

In [48]:
# Read in the data from S3 Buckets from Amazon
from pyspark import SparkFiles
# Load in Outdoor data from S3 into a DataFrame
url = "https://s3.amazonaws.com/amazon-reviews-pds/tsv/amazon_reviews_us_Outdoors_v1_00.tsv.gz"
spark.sparkContext.addFile(url)

df = spark.read.option('header', 'true').csv(SparkFiles.get("amazon_reviews_us_Outdoors_v1_00.tsv.gz"), inferSchema=True, sep='\t', timestampFormat="yyyy-mm-dd")
df.show(10)


+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-------------------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|        review_date|
+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-------------------+
|         US|   18446823|R35T75OLUGHL5C|B000NV6H94|     110804376|Stearns Youth Boa...|        Outdoors|          4|            0|          0|   N|                Y|          Four Stars|          GOOD VALUE|2015-01-31 00:08:00|
|         US|   13724367|R2BV735O46BN33|B000IN0W3Y|     624096774|Primal Wear Men's...| 

# TRANSFORM

## Counting Records: Drop duplicates and incomplete rows

In [49]:
print(df.count())
df = df.dropna()
print(df.count())
df = df.dropDuplicates()
print(df.count())

2302401
2302174
2302174


## Examine the schema

In [50]:
df.printSchema()

root
 |-- marketplace: string (nullable = true)
 |-- customer_id: integer (nullable = true)
 |-- review_id: string (nullable = true)
 |-- product_id: string (nullable = true)
 |-- product_parent: integer (nullable = true)
 |-- product_title: string (nullable = true)
 |-- product_category: string (nullable = true)
 |-- star_rating: integer (nullable = true)
 |-- helpful_votes: integer (nullable = true)
 |-- total_votes: integer (nullable = true)
 |-- vine: string (nullable = true)
 |-- verified_purchase: string (nullable = true)
 |-- review_headline: string (nullable = true)
 |-- review_body: string (nullable = true)
 |-- review_date: timestamp (nullable = true)



### SCHEMA TO REFER TO
CREATE TABLE review_id_table (
  review_id TEXT PRIMARY KEY NOT NULL,
  customer_id INTEGER,
  product_id TEXT,
  product_parent INTEGER,
  review_date DATE -- this should be in the formate yyyy-mm-dd
);

-- This table will contain only unique values
CREATE TABLE products (
  product_id TEXT PRIMARY KEY NOT NULL UNIQUE,
  product_title TEXT
);

-- Customer table for first data set
CREATE TABLE customers (
  customer_id INT PRIMARY KEY NOT NULL UNIQUE,
  customer_count INT
);

-- vine table
CREATE TABLE vine_table (
  review_id TEXT PRIMARY KEY,
  star_rating INTEGER,
  helpful_votes INTEGER,
  total_votes INTEGER,
  vine TEXT
);


In [51]:
# Change datatypes: 
# Source: https://stackoverflow.com/questions/32284620/how-to-change-a-dataframe-column-from-string-type-to-double-type-in-pyspark

#from pyspark.sql.types import TimestampType, IntegerType
# timestamp matched when loading
#  |-- review_body: string (nullable = true).   # review_date DATE
#customer_count INT. need to create

# sample code for changing
# dfc = df.withColumn("customer_id", df["customer_id"].cast(IntegerType()))\
#        .withColumn("review_date", df["review_date"].cast(TimestampType()))\



Review table

In [52]:
## Creater review_id_table DataFrame
review_id_table_df = df.select(["review_id","customer_id","product_id", "product_parent","review_date"])
review_id_table_df.show()

+--------------+-----------+----------+--------------+-------------------+
|     review_id|customer_id|product_id|product_parent|        review_date|
+--------------+-----------+----------+--------------+-------------------+
|R100AJS24RR6PQ|   44185913|B000IZX0SG|     373520218|2015-01-22 00:02:00|
|R100F6MBZKRAFD|   49266466|B00NCJPXEO|      80630966|2014-01-01 00:12:00|
|R100Y47MF2D3EF|   29788575|B001O5M1CA|     139791691|2013-01-27 00:05:00|
|R101EMNRQWQ4NA|   49971220|B00A2C8BXO|     949028516|2014-01-17 00:10:00|
|R101T5ACN6GE6Y|    1512429|B000NPCSU2|     435965836|2015-01-04 00:02:00|
|R102458UPCSCJF|   10518483|B000IZGF42|     100719823|2015-01-01 00:07:00|
|R102MVX3XAWLW8|   17049802|B009R9FUI2|     232637943|2013-01-28 00:05:00|
|R1030MR86HT0G5|   22406799|B0000AUSFB|     145114599|2013-01-02 00:02:00|
|R10358ROTFT3IB|     928913|B00LA97I8M|     165160843|2015-01-06 00:05:00|
|R103VYR9VYTJLB|   31778769|B000K00O1A|     267773760|2015-01-18 00:04:00|
|R10409ALR168X3|   320238

In [53]:
# Print schema
review_id_table_df.printSchema()

root
 |-- review_id: string (nullable = true)
 |-- customer_id: integer (nullable = true)
 |-- product_id: string (nullable = true)
 |-- product_parent: integer (nullable = true)
 |-- review_date: timestamp (nullable = true)



Products table

In [54]:
# Create product dataframe to match products table(only unique values)
products_df = df.select(["product_id","product_title"])
products_df = products_df.dropDuplicates()
products_df.show()


+----------+--------------------+
|product_id|       product_title|
+----------+--------------------+
|B00IFHFJXI|Ivation Portable ...|
|B00WG0J0D0|JanSport Superbre...|
|B00V15AUN0|Nickelodeon Paw P...|
|B00FUWSTI8|Bago Lightweight ...|
|B003FV94NA|Michelin Lithion ...|
|B00WIK04HO|Ultra Bright Camp...|
|B00J2HSCM0|High Sierra Tank ...|
|B009I6NSR4|Black Veil Brides...|
|B001GSHSLE|Stansport 191 App...|
|B00L2IO9M4|Columbia Sportswe...|
|B00KY7IM7W|Nalgene 32 Oz Wid...|
|B00TV5JCTK|Rollerblade ABEC ...|
|B00B9D071Y|BUFF UV Multifunc...|
|B00F9IGIKO|Condor Tactical F...|
|B004X55L9I|Hydro Flask Insul...|
|B00LORROIY|Scuba Choice Divi...|
|B00AATRU8G|Kelty Redwing 44 ...|
|B00HMCYWEO|Dakine Explorer L...|
|B004DK1CM8|Hot Headz 12V Hea...|
|B00T4W6SSS|Fits Sock Light H...|
+----------+--------------------+
only showing top 20 rows



Customers table

In [55]:
# Create customers dataframe to match customers table
from pyspark.sql.functions import desc

customers_df = df.groupby("customer_id").agg({"customer_id":"count"})
customers_df = customers_df.orderBy(desc("count(customer_id)"))
customers_df = customers_df.withColumnRenamed("count(customer_id)", "customer_count") 
customers_df.show()

+-----------+--------------+
|customer_id|customer_count|
+-----------+--------------+
|   13355404|           291|
|   47355039|           205|
|   51155788|           197|
|   20433453|           129|
|   40944345|           128|
|    3247563|           128|
|   27550730|           125|
|   30636778|           115|
|   10796373|           113|
|   16255502|           112|
|   40364490|           112|
|   38137849|           111|
|   52988261|           109|
|   36205928|           108|
|   18466687|           104|
|   52196389|           104|
|   44142946|           102|
|   49782074|           100|
|   45014035|           100|
|   31807575|            98|
+-----------+--------------+
only showing top 20 rows



Vine table

In [56]:
# Create vine_table DataFrame

# Load in a sql function to use columns
from pyspark.sql.functions import col

vine_table_df = df.select(["review_id","star_rating","helpful_votes", "total_votes","vine"])

#show
vine_table_df.show() 

+--------------+-----------+-------------+-----------+----+
|     review_id|star_rating|helpful_votes|total_votes|vine|
+--------------+-----------+-------------+-----------+----+
|R100AJS24RR6PQ|          2|            0|          0|   N|
|R100F6MBZKRAFD|          4|            0|          0|   N|
|R100Y47MF2D3EF|          5|            0|          0|   N|
|R101EMNRQWQ4NA|          5|            0|          0|   N|
|R101T5ACN6GE6Y|          5|            0|          0|   N|
|R102458UPCSCJF|          5|            0|          0|   N|
|R102MVX3XAWLW8|          1|           13|         66|   N|
|R1030MR86HT0G5|          4|            1|          1|   N|
|R10358ROTFT3IB|          5|            1|          1|   N|
|R103VYR9VYTJLB|          5|            0|          0|   N|
|R10409ALR168X3|          5|            0|          0|   N|
|R104F0FVV92T5I|          4|            1|          1|   N|
|R104RUECA3QHXR|          1|            0|          1|   N|
|R105AJ8IYCFIXJ|          5|            

In [ ]:
# DataFrame to RDS
# Configuration for RDS instance
mode="append"
jdbc_url = "jdbc:postgresql://<insert endpoint>:5432/<edit database name>"
config = {"user":"root",
          "password": "<insert password>",
          "driver":"org.postgresql.Driver"}

In [58]:
# Write DataFrame to RDS
review_id_table_df.write.jdbc(url=jdbc_url, table='review_id_table', mode=mode, properties=config)


In [20]:
# Write DataFrame to products table in RDS
products_df.write.jdbc(url=jdbc_url, table='products', mode=mode, properties=config)


In [59]:
# Write DataFrame to customers table in RDS
customers_df.write.jdbc(url=jdbc_url, table='customers', mode=mode, properties=config)


In [22]:
# Write DataFrame to vine_table table in RDS
vine_table_df.write.jdbc(url=jdbc_url, table='vine_table', mode=mode, properties=config)
